In [ ]:
import keras
import numpy as np 
import requests
import datetime
import pandas as pd
import pandas_datareader.data as reader
from sklearn.svm import SVR
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import style

def create(symbol,start,end):
    style.use('ggplot')
    df = reader.DataReader("AAPL", 'yahoo', start, end)
    # sort by date
    df = df.sort_values('Date') 
    # fix the date 
    df.reset_index(inplace=True)
    return(df)

def accessGrant():
    stripped = ""
    # adding noise to the data process:
    # data[::5] += 3 * (0.5 - np.random.rand(5))
    # accessfile = open("access.txt","r")
    with open("access.txt","r") as access:
            line = access.readline()
            cnt = 1
            while line:
                if ("finnhub" in line):
                    print(f'Line {cnt}: {line.strip("finnhub=~")}')
                stripped = line.strip("finnhub=~")
                line = access.readline()
                cnt += 1
                if(cnt>2):
                    break

def unixTimeStamp():
    timenow = datetime.datetime.now()
    epoch = datetime.datetime.utcfromtimestamp(0)
    value = (timenow - epoch).total_seconds() * 1000.0
    return(value,timenow)

def genWebHook():
    r = requests.post('https://finnhub.io/api/v1/webhook/add?token=', stripped , json={'event': 'earnings', 'symbol': 'AAPL'})
    res = r.json() # limit 30/sec base
    print(res)

def main():
    utc , _ = unixTimeStamp()
    print(utc)
    start = datetime.datetime(2019,8,5)
    end = datetime.datetime.today()
    df = create("AAPL",start,end)
    print(df.head())
    # genWebHook()
if __name__ == "__main__":
    main()

# quandl.ApiConfig.api_key = contents
# data = quandl.get('WIKI/TSLA', start_date='2019-12-26', end_date='2020-7-28')
# tsla = quandl.get("EOD/TSLA")
# print(tsla.head())

In [ ]:
df

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(df["Adj Close"])

In [ ]:
storeAdjClose = df['Adj Close']
mavg = storeAdjClose.rolling(window=100).mean()

In [ ]:
plt.figure(figsize = (12,5))
storeAdjClose.plot()

In [ ]:
plt.figure(figsize = (12,6))
storeAdjClose.plot(label='TSLA')
mavg.plot(label='mavg')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

In [ ]:

dates_df = df.copy()
dates_df = dates_df.reset_index()

org_dates = dates_df['Date']

dates_df['Date'] = dates_df['Date'].map(mdates.date2num)

dates_df.tail()

In [ ]:
# kernels are used to map lower dimention datapoints to a high dimentional data space
dates = dates_df['Date'].to_numpy()
prices = df['Adj Close'].to_numpy()
dates = np.reshape(dates, (len(dates), 1))
prices = np.reshape(prices, (len(prices), 1))
svrTickerLinear = SVR(kernel= 'linear')
svrTickerLinear.fit(dates, prices)
plt.figure(figsize = (12,6))
plt.plot(dates, prices, color= 'black', label= 'Data')
plt.plot(org_dates, svrTickerLinear.predict(dates), color= 'red', label= 'RBF model') 
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()


In [ ]:
svrTickerRBF = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1)
svrTickerRBF.fit(dates, prices)
plt.figure(figsize = (12,6))
plt.plot(dates, prices, color= 'black', label= 'Data')
plt.plot(org_dates, svrTickerRBF.predict(dates), color= 'red', label= 'RBF model') 
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
svrTicker = SVR(kernel= 'sigmoid')
svrTicker.fit(dates, prices)
plt.figure(figsize = (12,6))
plt.plot(dates, prices, color= 'black', label= 'Data')
plt.plot(org_dates, svrTicker.predict(dates), color= 'red', label= 'RBF model') 
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
style.use('ggplot')
start_b = datetime.datetime(2018,1,1)
end_b = datetime.datetime.today()
# start = datetime.datetime(2020,3,11)
# end = datetime.datetime(2020,8,1)
# end = datetime.date.today()
df_b = reader.DataReader("MCD", 'yahoo', start_b, end_b)
# sort by date
df_b = df_b.sort_values('Date')


In [ ]:
print(df_b)

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(df_b["Adj Close"])

In [1]:
import yfinance as yf

def visualizeYfinanceHistoricalData(symbol):
    ticker = yf.Ticker(symbol)
    data = ticker.history(start="2010-01-01",  end="2020-07-21")
    fig = plotit.Figure(data=[plotit.Candlestick(x=data['Date'],high=data['High'],low=data['Low'],close=data['Close'])])  
    fig.show() 

In [ ]:
visualizeYfinanceHistoricalData("F")